In [1]:
from Bagging import create_models, create_bags, evaluate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
import random
from BaggingSA import BaggingSA
from typing import Literal, Tuple
from Bagging import predict
import sklearn


In [ ]:
seed = 42

k_cross = 5
fitness_accuracy_diversity_ratios = [.25, .5, .75] 
feature_mutation_chances = [.1, .2, .3]
# datasets = ['wine', 'breast_cancer', 'pima', 'digits']
datasets = ['digits', 'pima']


np.random.seed(seed)
random.seed(seed)

In [3]:


def get_dataset(dataset_name: str) -> Tuple[np.ndarray, np.ndarray]:
    if dataset_name == 'digits':
        data = sklearn.datasets.load_digits()
        X = data.data
        y = data.target
        
    elif dataset_name == 'wine':
        data = sklearn.datasets.load_wine()
        X = data.data
        y = data.target
    
    elif dataset_name == 'breast_cancer':
        data = sklearn.datasets.load_breast_cancer()
        X = data.data
        y = data.target
        
    elif dataset_name == 'pima':
        data = pd.read_csv("./../datasets/pima.csv")
        X = data.iloc[:, :-1].values
        y = data.iloc[:, -1].values
    
    else:
        raise ValueError("Unsupported dataset")
    return X, y

In [4]:

def evaluate_bagging_sa(X_train, y_train, X_test, y_test, fitness_accuracy_diversity_ratio, feature_mutation_chance) -> Tuple[float, int, int]: 
    bagging_sa = BaggingSA(X=X_train, y=y_train,
                            T0=2.0, cooling_method='geometric', alpha=0.995, max_iterations=2000, n_trees=10,
                            fitness_accuracy_diversity_ratio=fitness_accuracy_diversity_ratio,
                            feature_mutation_chance=feature_mutation_chance, test_split_amount=20)
    models, fitness = bagging_sa.run(monitor_fun=fun_monitor, get_fitness=True, X_for_test=X_test, y_for_test=y_test)
    accuracy = evaluate(X=X_test, y=y_test, models=models)
    return bagging_sa, accuracy, fitness
    
def fun_monitor(iteration, T, best_fitness, fitness, new_fitness, accuracy):
    global fit_acc_sum, acc_fitness_difference
    
    acc_fitness_difference += abs(accuracy - fitness)
    
    if new_fitness not in fit_acc_sum:
        fit_acc_sum[new_fitness] = [accuracy]
    else:
        fit_acc_sum[new_fitness].append(accuracy)

    if iteration % 100 == 0:
        print(f"    Iteration: {iteration}, T: {T:.2f}, Best fitness: {best_fitness:.4f}")

acc_fitness_difference = 0.0
fit_acc_sum = {}
result = []
print(f"Start at {pd.Timestamp.now()}")
for dataset in datasets:
    X, y = get_dataset(dataset)       
    sub_groups_X = np.array_split(np.array(X), k_cross)
    sub_groups_y = np.array_split(np.array(y), k_cross) 
         
    for fadr in fitness_accuracy_diversity_ratios:
        for fmc in feature_mutation_chances:
            for k in range(k_cross):
                print(f"[Dataset: {dataset}, FADR: {fadr}, FMC: {fmc}, k: {k}]")
                
                X_train = np.concatenate(sub_groups_X[:k] + sub_groups_X[k+1:])
                y_train = np.concatenate(sub_groups_y[:k] + sub_groups_y[k+1:])
                X_test = sub_groups_X[k]
                y_test = sub_groups_y[k]
                
                acc_fitness_difference = 0.0
                fit_acc_sum = {}
                
                bagging_sa, accuracy, fitness = evaluate_bagging_sa(X_train, y_train, X_test, y_test, fadr, fmc)
                
                X = np.array(list(fit_acc_sum.keys()))
                Y = np.array([np.mean(fit_acc_sum[x]) for x in X])
                correlation = np.corrcoef(X, Y)[0, 1]
                
                acc_fitness_difference /= bagging_sa.max_iterations
                
                result.append([dataset, k, fadr, fmc, accuracy, correlation, fitness, acc_fitness_difference])
                
                df = pd.DataFrame(result, columns=["dataset", "kCrossIndex", "fadr", "fmc", "accuracy", "correlation", "fitness", "accFitnessDifference"])
                df.to_csv("./../res/bagging_sa_params.csv", index=False)
                print(f"    Accuracy: {accuracy:.4f}")

            
                
            
    

Start at 2025-04-15 12:11:42.775218
[Dataset: digits, FADR: 0.25, FMC: 0.1, k: 0]
Iteration: 100, T: 1.22, Best fitness: 0.9543
Iteration: 200, T: 0.74, Best fitness: 0.9635
Iteration: 300, T: 0.45, Best fitness: 0.9724
Iteration: 400, T: 0.27, Best fitness: 0.9724
Iteration: 500, T: 0.16, Best fitness: 0.9740
Iteration: 600, T: 0.10, Best fitness: 0.9740
Iteration: 700, T: 0.06, Best fitness: 0.9740
Iteration: 800, T: 0.04, Best fitness: 0.9740
Iteration: 900, T: 0.02, Best fitness: 0.9740
Iteration: 1000, T: 0.01, Best fitness: 0.9740
Iteration: 1100, T: 0.01, Best fitness: 0.9740
Iteration: 1200, T: 0.00, Best fitness: 0.9740
Iteration: 1300, T: 0.00, Best fitness: 0.9740
Iteration: 1400, T: 0.00, Best fitness: 0.9811
Iteration: 1500, T: 0.00, Best fitness: 0.9844
Iteration: 1600, T: 0.00, Best fitness: 0.9849
Iteration: 1700, T: 0.00, Best fitness: 0.9849
Iteration: 1800, T: 0.00, Best fitness: 0.9849
Iteration: 1900, T: 0.00, Best fitness: 0.9849
Iteration: 2000, T: 0.00, Best fit

KeyboardInterrupt: 

In [5]:
df = pd.read_csv("./../res/bagging_sa_params.csv")
df_agg = df.groupby(["dataset", "fadr", "fmc"]).agg(
    dataset=("dataset", "first"),
    fadr=("fadr", "first"),
    fmc=("fmc", "first"),
    accuracyMean=("accuracy", "mean"),
    accuracyStd=("accuracy", "std"),
    correlationMean=("correlation", "mean"),
    correlationStd=("correlation", "std"),
    fitnessMean=("fitness", "mean"),
    fitnessStd=("fitness", "std"),
    accFitnessDifferenceMean=("accFitnessDifference", "mean"),
    accFitnessDifferenceStd=("accFitnessDifference", "std"),
).reset_index(drop=True)
df_agg.to_csv("./../res/bagging_sa_params_aggregated.csv", index=False)

print(df_agg.head())

  dataset  fadr  fmc  accuracyMean  accuracyStd  correlationMean  \
0  digits  0.25  0.1         0.875     0.027499         0.261226   

   correlationStd  fitnessMean  fitnessStd  accFitnessDifferenceMean  \
0         0.16773     0.982706    0.003168                  0.121549   

   accFitnessDifferenceStd  
0                 0.046606  
